In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import pathlib

In [ ]:
!unzip "/content/drive/MyDrive/sign_data.zip"

In [ ]:
data = pathlib.Path('/content/data')


In [ ]:
image_count = len(list(data.glob('*/*.jpg')))
print(image_count)

In [ ]:
batch_size = 32
img_height = 90
img_width = 90

In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
  data,
  validation_split=0.2,
  subset="training",
  seed=7,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_data = tf.keras.preprocessing.image_dataset_from_directory(
  data,
  validation_split=0.2,
  subset="validation",
  seed=7,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_data.class_names
print(class_names)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_data.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_data:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = 35
model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [ ]:
model.fit(train_data,validation_data=val_data,epochs=1)

In [ ]:
model.summary()

In [ ]:
model.save('Sign_CNN.h5')

In [ ]:
model.load_weights('Sign_CNN.h5')